In [2]:
db_name = 'ted-talks/ted-talks.db'

In [3]:
OHCO = ['speaker', 'event', 'id', 'para_num', 'sent_num', 'token_num']
AUTHS = OHCO[:1]
BOOKS = OHCO[:2]
CHAPS = OHCO[:3]
PARAS = OHCO[:4]
SENTS = OHCO[:5]

BAG = CHAPS

In [4]:
import pandas as pd
import numpy as np
import sqlite3

import sys
from gensim.models import word2vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

### Pragmas

In [5]:
%matplotlib inline

In [6]:
with sqlite3.connect(db_name) as db:
    tokens = pd.read_sql('SELECT * FROM token', db, index_col=OHCO)
    vocab = pd.read_sql('SELECT * FROM vocab', db, index_col='term_id')
    docs =  pd.read_sql('SELECT * FROM doc', db, index_col=CHAPS)

In [7]:
tokens.head()

pos  \
speaker     event                    id    para_num sent_num token_num        
 Crush Club TED Salon: Radical Craft 35945 1        0        0          NNP   
                                                             1          NNP   
                                                             2          NNP   
                                                             3          PRP   
                                                             4          VBG   

                                                                       token_str  \
speaker     event                    id    para_num sent_num token_num             
 Crush Club TED Salon: Radical Craft 35945 1        0        0                TC   
                                                             1            Milan:   
                                                             2               How   
                                                             3               you   
                                                             4             doing   

                                                                        punc  \
speaker     event                    id    para_num sent_num token_num         
 Crush Club TED Salon: Radical Craft 35945 1        0        0             0   
                                                             1             0   
                                                             2             0   
                                                             3             0   
                                                             4             0   

                                                                        num  \
speaker     event                    id    para_num sent_num token_num        
 Crush Club TED Salon: Radical Craft 35945 1        0        0            0   
                                                             1            0   
                                                             2            0   
                                                             3            0   
                                                             4            0   

                                                                       term_str  \
speaker     event                    id    para_num sent_num token_num            
 Crush Club TED Salon: Radical Craft 35945 1        0        0               tc   
                                                             1            milan   
                                                             2              how   
                                                             3              you   
                                                             4            doing   

                                                                        term_id  
speaker     event                    id    para_num sent_num token_num           
 Crush Club TED Salon: Radical Craft 35945 1        0        0            65498  
                                                             1            41011  
                                                             2            30566  
                                                             3            74082  
                                                             4            18342

In [8]:
tokens = tokens[~tokens.term_str.isna()]
corpus = tokens.groupby(OHCO).term_str.apply(lambda  x:  x.tolist()).reset_index()['term_str'].tolist()

In [11]:
corpus[:5]

[['tc'], ['milan'], ['how'], ['you'], ['doing']]

## Generate word embeddings with Gensim's library

In [ ]:
model = word2vec.Word2Vec(corpus, size=246, window=5, min_count=200, workers=4)

## Visualize with tSNE

In [ ]:
coords = pd.DataFrame(index=range(len(model.wv.vocab)))
coords['label'] = [w for w in model.wv.vocab]
coords['vector'] = coords['label'].apply(lambda x: model.wv.get_vector(x))

In [ ]:
tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
tsne_values = tsne_model.fit_transform(coords['vector'].tolist())

In [ ]:
coords['x'] = tsne_values[:,0]
coords['y'] = tsne_values[:,1]

In [ ]:
coords.head()

In [ ]:
plt.figure(figsize=(16, 16)) 
for i in range(len(coords)):
    plt.scatter(coords.x[i],coords.y[i])
    plt.annotate(word_labels[i],
                 xy=(coords.x[i], coords.y[i]),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
plt.show()